In [20]:
! pip install llama-index langchain llama-index-llms-openai-like llama-index-embeddings-huggingface
! pip install EbookLib html2text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 1.2 MB/s eta 0:00:001.5 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 5.9 MB/s eta 0:00:000:00:010:00:01:01
  Created wheel for EbookLib: filename=EbookLib-0.18-py3-none-any.whl size=38778 sha256=f92650745984b8ae606359504aff9185159ed0d3b005c846c923c3f00b8568b8
  Stored in directory: /Users/ezonghu/Library/Caches/pip/wheels/0f/38/cc/a3728bb72a315d9d8766fb71d362136372066fc25ad838f8fa
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33110 sha256=31344728d0e4120c0fe4203d7768a17e25dd5426975250d49cbbcbe09180084d
  Stored in directory: /Users/ezonghu/Library/Caches/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built EbookLib html2text


# 使用Qwen 1.5 14B int4 模型

## Llama-index与langchain调用本地模型提供的类OpenAi restful接口

In [1]:
from llama_index.core.llms import  ChatMessage
from llama_index.llms.openai_like import OpenAILike


# from llama_index. import ChatMessage, OpenAILike  

li_llm = OpenAILike(  
    api_base="http://localhost:8000/",  
    timeout=600,  # secs  
    api_key="loremIpsum",  
    is_chat_model=True,  
    context_window=32768,  
)  
chat_history = [  
    ChatMessage(role="system", content="你是韩立"),  
    ChatMessage(role="user", content="你的老婆是谁？"),  
]  
output = li_llm.chat(chat_history)  
print(output)

/opt/homebrew/anaconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


assistant: 在修真小说《凡人修仙传》中，我没有提及具体的婚姻关系，因为我的身份是韩立，一个专注于修炼的修士，并非世俗之人。所以，我并没有老婆这一角色设定。我的故事主要围绕着个人的修行和冒险展开。


In [3]:
from urllib import response
from langchain.schema import HumanMessage, SystemMessage  
from langchain_openai import ChatOpenAI  
import pprint
  
lc_llm = ChatOpenAI(  
    openai_api_base="http://localhost:8000/",  
    request_timeout=600,  # secs, I guess.  
    openai_api_key="loremIpsum",  
    max_tokens=32768,  
)  
chat_history = [  
    SystemMessage(content="你是韩立"),  
    HumanMessage(content="你的老婆是谁？"),  
]  
response = lc_llm(chat_history)
print(response.content)

我是韩立，修真小说中的角色，并非现实世界的人物，所以我没有老婆。我的故事围绕在修炼和冒险之中。


In [4]:
from llama_index.core import ServiceContext, SimpleDirectoryReader, VectorStoreIndex


documents = SimpleDirectoryReader(
    input_dir="/Users/ezonghu/Downloads/fanren",
).load_data()  
index = VectorStoreIndex.from_documents(documents)

/opt/homebrew/anaconda3/envs/llm/lib/python3.10/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}